In [11]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.graphs import Neo4jGraph


os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

index_name = 'typical_rag'


store = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASSWORD'),
    index_name=index_name,
)

## Graph model

<img src="img/graph_model.png" width="800"/>

### Vector similarity search

In [12]:
def do_similarity_search(store,question,k):
    docs_with_score = store.similarity_search_with_score(question, k=k)
    for doc, score in docs_with_score:
        print("-" * 120)
        print("Score: ", score)
        print(doc.page_content)

In [13]:
question = """How does Harry Potter discover he is a wizard?"""
do_similarity_search(store,question,k=2)

------------------------------------------------------------------------------------------------------------------------
Score:  0.9279569983482361
, just before Harry's eleventh birthday, owls begin delivering letters addressed to him. When the abusive Dursleys adamantly refuse to allow Harry to open any and flee to an island hut, Hagrid arrives to personally deliver Harry's letter of acceptance to Hogwarts. Hagrid also reveals that Harry's parents, James and Lily, were killed by a dark wizard named Lord Voldemort. The killing curse that Voldemort had cast towards Harry rebounded, destroying Voldemort's body and giving Harry his lightning-bolt scar. Hagrid then takes Harry to Diagon Alley for school supplies and gives him a pet snowy owl whom he names Hedwig. Harry buys a wand that is connected to Voldemort's own wand.
At King's Cross station, Harry boards the Hogwarts Express train, and meets fellow students Ron Weasley and Hermione Granger during the journey. Arriving at Hogwarts, H

In [102]:
question = "What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?"
do_similarity_search(store,question,k=2)

------------------------------------------------------------------------------------------------------------------------
Score:  0.9430073499679565
Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone) is a 2001 fantasy film directed by Chris Columbus and produced by David Heyman, from a screenplay by Steve Kloves, based on the 1997 novel of the same name by J. K. Rowling. It is the first instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its story follows Harry's first year at Hogwarts School of Witchcraft and Wizardry as he discovers that he is a famous wizard and begins his formal wizarding education.
Warner Bros. Pictures bought the film rights to the book in 1999 for a reported £1 million ($1.65 million). Production began in the United Kingdom in 2000, with Columbus being chosen to helm the film from a short list of directors tha

##### Q&A with typical_rag index (typical chunking strategy)

In [14]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [5]:
def qna(question,k):
    template = """You are a helpful assistant. Your task is to answer the question using only the provided document.
    {context}

    Question: {question}
    Answer:
    """
    prompt = PromptTemplate(
    template=template,
    input_variables=[
        'context',
        'question',
    ]
)
    child_store = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASSWORD'),
    index_name="typical_rag",
)
    vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0,model_name=os.getenv('OPENAI_MODEL')), chain_type="stuff", retriever=child_store.as_retriever(search_kwargs={"k":k}),chain_type_kwargs={
        "prompt": prompt
    })
    print("Document list : ")
    do_similarity_search(child_store,question,k=k)
    print("\n\n\nAnswer is : ")
    return vector_qa.run(question)

In [15]:
print(qna(question="How does Harry Potter discover he is a wizard?",k=2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9279569983482361
, just before Harry's eleventh birthday, owls begin delivering letters addressed to him. When the abusive Dursleys adamantly refuse to allow Harry to open any and flee to an island hut, Hagrid arrives to personally deliver Harry's letter of acceptance to Hogwarts. Hagrid also reveals that Harry's parents, James and Lily, were killed by a dark wizard named Lord Voldemort. The killing curse that Voldemort had cast towards Harry rebounded, destroying Voldemort's body and giving Harry his lightning-bolt scar. Hagrid then takes Harry to Diagon Alley for school supplies and gives him a pet snowy owl whom he names Hedwig. Harry buys a wand that is connected to Voldemort's own wand.
At King's Cross station, Harry boards the Hogwarts Express train, and meets fellow students Ron Weasley and Hermione Granger during the journey. Arrivi

In [16]:
print(qna(question="What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?",k=2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9430073499679565
Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone) is a 2001 fantasy film directed by Chris Columbus and produced by David Heyman, from a screenplay by Steve Kloves, based on the 1997 novel of the same name by J. K. Rowling. It is the first instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its story follows Harry's first year at Hogwarts School of Witchcraft and Wizardry as he discovers that he is a famous wizard and begins his formal wizarding education.
Warner Bros. Pictures bought the film rights to the book in 1999 for a reported £1 million ($1.65 million). Production began in the United Kingdom in 2000, with Columbus being chosen to helm the film from a short list

In [18]:
print(qna(question="Which actors appeared in all eight Harry Potter films, and who are their characters?",k=10))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9229376316070557
ry.
Rupert Grint as Ron Weasley: Harry's best friend at Hogwarts and a younger member of the Weasley wizarding family.
Emma Watson as Hermione Granger: Harry's other best friend and the trio's brains.
Robbie Coltrane as Rubeus Hagrid: The gamekeeper and new Care of Magical Creatures teacher at Hogwarts.
Michael Gambon as Albus Dumbledore:The headmaster of Hogwarts and one of the greatest wizards of all time. Gambon assumed the role after Richard Harris, who played Dumbledore in the previous two films, died of Hodgkin's disease on 25 October 2002, three weeks before the second film's release. Despite his illness, Harris was determined to film his part, telling a visiting David Heyman not to recast the role. Four months after Harris's death, Cuarón chose Gambon as his replacement. Gambon was unconcerned with bettering or copy

#### Harry Potter: 10 Supporting Characters Who Appear Most Frequently In The Movies


<img src="img/Minerva%20McGonagal.png" width="500"/>



ref : https://screenrant.com/harry-potter-supporting-characters-appearance-frequency-ranked/#minerva-mcgonagall

## Graph model

<img src="img/graph_model.png" width="800"/>

##### Let's try another RAG strategy to improve its accuracy

In [20]:
def qna_child(question,k):
    template = """You are a helpful assistant. Your task is to answer the question using only the provided document.
    {context}

    Question: {question}
    Answer:
    """
    prompt = PromptTemplate(
    template=template,
    input_variables=[
        'context',
        'question',
    ]
)
    child_store = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASSWORD'),
    index_name="child_document",

)
    vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0,model_name=os.getenv('OPENAI_MODEL')), chain_type="stuff", retriever=child_store.as_retriever(search_kwargs={"k":k}),chain_type_kwargs={
        "prompt": prompt
    })
    print("Document list : ")
    do_similarity_search(child_store,question,k=k)
    print("\n\n\nAnswer is : ")
    return vector_qa.run(question)

In [21]:
print(qna_child(question="How does Harry Potter discover he is a wizard?",k=2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9349277019500732
. The killing curse that Voldemort had cast towards Harry rebounded, destroying Voldemort's body and giving Harry his lightning-bolt scar. Hagrid then takes Harry to Diagon Alley for school supplies and gives him a pet snowy owl whom he names Hedwig. Harry buys a wand that is connected to Voldemort's own wand.
At King's Cross station, Harry boards the Hogwarts Express train, and meets fellow students Ron Weasley and Hermione Granger during the journey. Arriving at Hogwarts, Harry also meets Draco
------------------------------------------------------------------------------------------------------------------------
Score:  0.9347399473190308

Harry discovers that his used Potions textbook is filled with helpful notes and spells added by the "Half-Blood Prince". Using it, Harry excels in class, annoying Hermione and impressi

In [106]:
print(qna_child(question="How does Harry Potter discover he is a wizard?",k=6))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9349277019500732
. The killing curse that Voldemort had cast towards Harry rebounded, destroying Voldemort's body and giving Harry his lightning-bolt scar. Hagrid then takes Harry to Diagon Alley for school supplies and gives him a pet snowy owl whom he names Hedwig. Harry buys a wand that is connected to Voldemort's own wand.
At King's Cross station, Harry boards the Hogwarts Express train, and meets fellow students Ron Weasley and Hermione Granger during the journey. Arriving at Hogwarts, Harry also meets Draco
------------------------------------------------------------------------------------------------------------------------
Score:  0.9347399473190308

Harry discovers that his used Potions textbook is filled with helpful notes and spells added by the "Half-Blood Prince". Using it, Harry excels in class, annoying Hermione and impressi

In [107]:
print(qna_child(question="What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?",k=2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9424055814743042
Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone) is a 2001 fantasy film directed by Chris Columbus and produced by David Heyman, from a screenplay by Steve Kloves, based on the 1997 novel of the same name by J. K. Rowling. It is the first instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its
------------------------------------------------------------------------------------------------------------------------
Score:  0.9423733949661255
 stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its story follows Harry's first year at Hogwarts School of Witchcraft and Wizardry as he discovers that he is a famous wiz

In [22]:
print(qna_child(question="Which actors appeared in all eight Harry Potter films, and who are their characters?",k=10))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9264693856239319
.Several actors from the previous film reprise their roles in Goblet of Fire. James and Oliver Phelps play Fred and George Weasley, Ron's twin brothers, and Bonnie Wright as their sister Ginny, while Mark Williams plays their father, Arthur Weasley. Tom Felton portrays Lucius Malfoy's son Draco, Harry's rival in Slytherin, while Jamie Waylett and Joshua Herdman appear as Crabbe and Goyle, Draco's minions. Matthew Lewis, Devon Murray and Alfred Enoch play Neville
------------------------------------------------------------------------------------------------------------------------
Score:  0.9264634847640991


Daniel Radcliffe as Harry Potter: A 17-year-old British wizard.
Rupert Grint as Ron Weasley: One of Harry's best friends.
Emma Watson as Hermione Granger: Harry's other best friend.
Helena Bonham Carter as Bellatrix Le

#### Summary content

In [23]:
print(qna("summarise plot of Harry Potter and the Order of the Phoenix",2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9441704750061035
Harry Potter and the Order of the Phoenix is a 2007 fantasy film directed by David Yates from a screenplay by Michael Goldenberg, based on the 2003 novel of the same name by J. K. Rowling. It is the sequel to Harry Potter and the Goblet of Fire (2005) and the fifth instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger. Its story follows Harry's fifth year at Hogwarts School of Witchcraft and Wizardry as the Ministry of Magic is in denial of Lord Voldemort's return.
Filming took place in England and Scotland for exterior locations and Leavesden Film Studios in Watford for interior locations from February to November 2006, with a one-month break in June. Post-production on the film continued f

In [110]:
print(qna_child("summarise plot of Harry Potter and the Order of the Phoenix",2))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9396058320999146
Harry Potter and the Order of the Phoenix is a 2007 fantasy film directed by David Yates from a screenplay by Michael Goldenberg, based on the 2003 novel of the same name by J. K. Rowling. It is the sequel to Harry Potter and the Goblet of Fire (2005) and the fifth instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint and Emma Watson as Harry's best friends Ron Weasley and Hermione Granger. Its story
------------------------------------------------------------------------------------------------------------------------
Score:  0.9320425987243652
 coming into his own in the same way that Harry is, we tried to get that into the film in other ways, as much as possible. So, you feel like, if not the details of that story, at least the spirit of it is present in the f

## Filter before sending the context to the LLM.

In [134]:
def qna_child_filter(question,k):
    template = """You are a helpful assistant. Your task is to answer the question using only the provided document.
    {context}

    Question: {question}
    Answer:
    """
    prompt = PromptTemplate(
    template=template,
    input_variables=[
        'context',
        'question',
    ])
    query = """MATCH (node:Child)
                where toLower(node.id) CONTAINS toLower("{}")
                WITH node as node,max(score) as score
                RETURN node.text AS text,
                score,
                node {{.*, embedding: Null}} AS metadata """.format("Harry Potter and the Philosopher's Stone")

    child_store = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASSWORD'),
    index_name="child_document",
    retrieval_query=query
    )
    vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0,model_name=os.getenv('OPENAI_MODEL')), chain_type="stuff", retriever=child_store.as_retriever(search_kwargs={"k":k}),chain_type_kwargs={
        "prompt": prompt
    })



    print("Document list : ")
    do_similarity_search(child_store,question,k=k)
    print("\n\n\nAnswer is : ")
    return vector_qa.run(question)

In [136]:
print(qna_child(question="What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?",k=10))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9423828721046448
Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone) is a 2001 fantasy film directed by Chris Columbus and produced by David Heyman, from a screenplay by Steve Kloves, based on the 1997 novel of the same name by J. K. Rowling. It is the first instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its
------------------------------------------------------------------------------------------------------------------------
Score:  0.9423558115959167
 stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its story follows Harry's first year at Hogwarts School of Witchcraft and Wizardry as he discovers that he is a famous wiz

In [137]:
print(qna_child_filter(question="What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?",k=10))

Document list : 
------------------------------------------------------------------------------------------------------------------------
Score:  0.9423828721046448
Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone) is a 2001 fantasy film directed by Chris Columbus and produced by David Heyman, from a screenplay by Steve Kloves, based on the 1997 novel of the same name by J. K. Rowling. It is the first instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its
------------------------------------------------------------------------------------------------------------------------
Score:  0.9423558115959167
 stars Daniel Radcliffe as Harry Potter, with Rupert Grint as Ron Weasley, and Emma Watson as Hermione Granger. Its story follows Harry's first year at Hogwarts School of Witchcraft and Wizardry as he discovers that he is a famous wiz

In [128]:
print(qna_child_filter("summarise plot of Harry Potter and the Order of the Phoenix",3))

Document list from similarity search : 
------------------------------------------------------------------------------------------------------------------------
Score:  1.0
 Phoenix for Best Feature Film in 2007 and the Hugo Awards nominated the film for Best Dramatic Presentation (Long Form) in 2008.


== References ==


== External links ==

Official website
Harry Potter and the Order of the Phoenix at IMDb
Harry Potter and the Order of the Phoenix at AllMovie

------------------------------------------------------------------------------------------------------------------------
Score:  0.9396058320999146
Harry Potter and the Order of the Phoenix is a 2007 fantasy film directed by David Yates from a screenplay by Michael Goldenberg, based on the 2003 novel of the same name by J. K. Rowling. It is the sequel to Harry Potter and the Goblet of Fire (2005) and the fifth instalment in the Harry Potter film series. The film stars Daniel Radcliffe as Harry Potter, alongside Rupert Grint an